# 操作前提

安装和部署MinIO请参考README.md

## 安装Nuget包

In [ ]:
#r "nuget: AWSSDK.S3, 3.7.1.21"

Installed Packages AWSSDK.S3, 3.7.1.21

## 导入命名空间

In [ ]:
using System;
using System.Threading.Tasks;
using Amazon;
using Amazon.Util;
using Amazon.S3;
using Amazon.S3.Model;
using Amazon.S3.Util;
using System.IO;


## 确认访问信息

1. 服务器地址在docker-compose可知
2. 管理门户地址在docker-compose可知
3. 管理员用户名和密码在docker-compose.yaml中可知
3. 访问Key、密码Key可在管理门户里面申请



## 0. 公共定义和准备

In [ ]:
//REF: https://docs.aws.amazon.com/sdkfornet/v3/apidocs/items/Amazon/TRegionEndpoint.html
//why?
var regionEndpoint=RegionEndpoint.USEast1;
var serviceURL = "http://localhost:9000";

//TIPS: you may need use your own accessKey and secretKey which can get from http://127.0.0.1:9001/account
// var accessKey = "ASPSJQ1AJTTNGUXVBTWW";
// var secretKey = "Dvaq2QFhCWDgM+SxE9GXAd0ZqW8LSQoKNG0Ouwqh"; // do not store secret key hardcoded in your production source code!

var accessKey = "minio";
var secretKey = "minio123"; // do not store secret key hardcoded in your production source code!



var bucketName= "mytestbucket";

var config = new AmazonS3Config
    {
        RegionEndpoint = regionEndpoint, // MUST set this before setting ServiceURL and it should match the `MINIO_REGION` environment variable.
        ServiceURL = serviceURL, // replace http://localhost:9000 with URL of your MinIO server
        ForcePathStyle = true // MUST be true to work correctly with MinIO server
    };

//关于自动配置，可以参考 https://docs.aws.amazon.com/sdkfornet1/latest/apidocs/html/M_Amazon_S3_AmazonS3Client__ctor.htm
var amazonS3Client = new AmazonS3Client(accessKey, secretKey, config);


In [ ]:
//remove bucket if needed
//amazonS3Client.DeleteBucket(bucketName);
if(await AmazonS3Util.DoesS3BucketExistAsync(amazonS3Client, bucketName)){
    Console.WriteLine($"try to delete bucket {bucketName}");
    await AmazonS3Util.DeleteS3BucketWithObjectsAsync(amazonS3Client, bucketName);
}
else
{
    Console.WriteLine($"bucket:{bucketName} does not exist");
}


bucket:mytestbucket does not exist



## 1. create bucket

In [ ]:
if (!(await AmazonS3Util.DoesS3BucketExistAsync(amazonS3Client, bucketName)))
{
    Console.WriteLine($"Bucket:{bucketName} does not exists, try to create one");
    var putBucketRequest = new PutBucketRequest
    {
        BucketName = bucketName,
        UseClientRegion = true
    };

    PutBucketResponse putBucketResponse = await amazonS3Client.PutBucketAsync(putBucketRequest);
}
else
{
    Console.WriteLine("Bucket already exists");
}
// Retrieve the bucket location.

Bucket:mytestbucket does not exists, try to create one



## 2. create object

how to check succ or not?

"To ensure an object is not corrupted over the network, you can calculate the MD5 of an object, PUT it to Amazon S3, and compare the returned Etag to the calculated MD5 value."

ref: [amazon-s3-putobject-return-value-to-confirm-success](https://stackoverflow.com/questions/12454305/amazon-s3-putobject-return-value-to-confirm-success/12454592)

In [ ]:
// Warning: it seems AWS S3 & MINIO does not follow the same ETag format. especialy,
// 1. MINIO use low-case ETag.
// 2. MINIO ETag does not base64 encode.

// 1. Put object-specify only key name for the new object.
var content1="sample text 1"+DateTime.Now.ToString();
var content1_md5=AmazonS3Util.GenerateChecksumForContent(content1,false);
var putRequest1 = new PutObjectRequest
{
    BucketName = bucketName,
    Key = "text_file1.txt",
    ContentBody = content1,    
    //ContentBody = content1,
};
PutObjectResponse response1 = await amazonS3Client.PutObjectAsync(putRequest1);
Console.WriteLine($"MD5: {content1_md5}, ETag: {response1.ETag}");
if(response1.ETag.Trim('"').ToLower()==content1_md5.ToLower())
{
    Console.WriteLine("put object success");
}
else
{
    Console.WriteLine("put object failed");
}


// 2. Put the object-set ContentType and add metadata.
var content2="sample text 3"+DateTime.Now.ToString();
//CryptoUtilFactory.CryptoInstance.
var content2_md5=AmazonS3Util.GenerateChecksumForContent(content2,false);
File.WriteAllText(@"text_file2.txt", content2);
var putRequest2 = new PutObjectRequest
{
    BucketName = bucketName,
    Key = "text_file2.txt",
    FilePath = "text_file2.txt",
    ContentType = "text/plain"
};

putRequest2.Metadata.Add("x-amz-meta-title", "someTitle"+new Random().NextDouble());
PutObjectResponse response2 = await amazonS3Client.PutObjectAsync(putRequest2);
Console.WriteLine($"MD5: {content2_md5}, ETag: {response2.ETag}");
if(response2.ETag.Trim('"').ToLower()==content2_md5.ToLower())
{
    Console.WriteLine("put object success");
}
else
{
    Console.WriteLine("put object failed");
}

File.Delete(@"text_file2.txt");


// 3. Put object by stream
var content3="sample text 1"+DateTime.Now.ToString();
var content3_memory_stream=new MemoryStream(System.Text.Encoding.UTF8.GetBytes(content3));
//var content3_md5=AmazonS3Util.GenerateMD5ChecksumForStream(content3_memory_stream);
var hashed=CryptoUtilFactory.CryptoInstance.ComputeMD5Hash(content3_memory_stream);
var content3_md5=Convert.ToHexString(hashed);
content3_memory_stream.Seek(0,SeekOrigin.Begin);
var putRequest3 = new PutObjectRequest
{
    BucketName = bucketName,
    Key = "text_file3.txt",
    InputStream = content3_memory_stream,
    ContentType = "text/plain"
    //ContentBody = content1,
};
PutObjectResponse response3 = await amazonS3Client.PutObjectAsync(putRequest3);
Console.WriteLine($"MD5: {content3_md5}, ETag: {response3.ETag}");
if(response3.ETag.Trim('"').ToLower()==content3_md5.ToLower())
{
    Console.WriteLine("put object success");
}
else
{
    Console.WriteLine("put object failed");
}


MD5: 6890E6F2F57AF72A4230B36114C92FA7, ETag: "6890e6f2f57af72a4230b36114c92fa7"
put object success
MD5: BF8EA3DFA4B3FBA5AD8BE90057833973, ETag: "bf8ea3dfa4b3fba5ad8be90057833973"
put object success
MD5: 6890E6F2F57AF72A4230B36114C92FA7, ETag: "6890e6f2f57af72a4230b36114c92fa7"
put object success



## 3. list content of all object in all bucket 

In [ ]:
var listBucketResponse = await amazonS3Client.ListBucketsAsync();

foreach (var bucket in listBucketResponse.Buckets)
{
    Console.Out.WriteLine("bucket '" + bucket.BucketName + "' created at " + bucket.CreationDate);

    var bucketName = bucket.BucketName;

    var listObjectsResponse = await amazonS3Client.ListObjectsAsync(bucketName);

    foreach (var obj in listObjectsResponse.S3Objects)
    {
        Console.Out.WriteLine("\t"+"key = '" + obj.Key + "' | size = " + obj.Size + " | tags = '" + obj.ETag + "' | modified = " + obj.LastModified);

        GetObjectRequest request = new GetObjectRequest
        {
            BucketName = bucketName,
            Key = obj.Key
        };
        using (GetObjectResponse response = await amazonS3Client.GetObjectAsync(request))
        using (Stream responseStream = response.ResponseStream)
        using (StreamReader reader = new StreamReader(responseStream))
        {
            string title = response.Metadata["x-amz-meta-title"]; // Assume you have "title" as medata added to the object.
            string contentType = response.Headers["Content-Type"];
            Console.WriteLine("\t\t"+"Object metadata, Title: {0}", title);
            Console.WriteLine("\t\t"+"Content type: {0}", contentType);

            var responseBody = reader.ReadToEnd(); // Now you process the response body.
            Console.Out.WriteLine("\t\t"+"Content:"+responseBody);
        }

    }
}

bucket 'mytestbucket' created at 2021/8/10 下午11:47:01
	key = 'text_file1.txt' | size = 37 | tags = '"6890e6f2f57af72a4230b36114c92fa7"' | modified = 2021/8/10 下午11:47:02
		Object metadata, Title: 
		Content type: text/plain
		Content:sample text 12021/8/10 下午11:47:02
	key = 'text_file2.txt' | size = 37 | tags = '"bf8ea3dfa4b3fba5ad8be90057833973"' | modified = 2021/8/10 下午11:47:02
		Object metadata, Title: someTitle0.9806078304446338
		Content type: text/plain
		Content:sample text 32021/8/10 下午11:47:02
	key = 'text_file3.txt' | size = 37 | tags = '"6890e6f2f57af72a4230b36114c92fa7"' | modified = 2021/8/10 下午11:47:02
		Object metadata, Title: 
		Content type: text/plain
		Content:sample text 12021/8/10 下午11:47:02


## 自由练习区域

In [ ]:
string u="Hello World";
string l="hello world";
Console.WriteLine($"u:{u}, l:{l}, eq:{u.ToLower()==l}");

u:Hello World, l:hello world, eq:True


# 参考
1. https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html
2. https://docs.aws.amazon.com/sdkfornet/v3/apidocs/index.html